Нейрокопирайтер: видео с ютуб, аудио-файл и текстовый документ. Транскрибация видео и аудио, объединение текстов

In [ ]:
!pip install git+https://github.com/ytdl-org/youtube-dl.git
!pip install git+https://github.com/openai/whisper.git
!pip install tiktoken openai langchain langchain-community langchain_openai faiss-cpu translate gdown

In [25]:
import os
import time
from IPython.display import HTML, clear_output, display
import youtube_dl
import gdown
import datetime
import tiktoken
import whisper
from langchain.text_splitter import MarkdownHeaderTextSplitter, Document, RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from openai import OpenAI
from google.colab import userdata


clear_output()
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
client = OpenAI()
embeddings = OpenAIEmbeddings()

In [ ]:
system = """You are a text, copywriting, writing genius and a Python expert. Your job is to recognize
sections in a text and break it down into those parts, keeping the text 100% intact."""
user = """Please let's think step by step: Think about what sections in the text you can recognize and
what name you can give to each section in terms of meaning. Next, write your answer across the previous
answer in the form and order of: ## Title of section, followed by all text related to that section.
All in Russian. Text:"""
temperature = 0.3
chunk_size = 1000

# Функция настройки стиля для переноса текста в выводе ячеек
# для изменения стиля отображения текста, так чтобы предотвратить
# переполнение текста за границы ячейки вывода и обеспечить его перенос.
def set_text_wrap_css():
    css = '''
    <style>
    pre {
        white-space: pre-wrap;
    }
    </style>
    '''
    display(HTML(css))

get_ipython().events.register('pre_run_cell', set_text_wrap_css)

# Функция подсчета количества токенов
def num_tokens_from_messages(messages, model='gpt-3.5-turbo-0125'):
    try:
        encoding = tiktoken.encoding_for_model(model)
    except KeyError:
        encoding = tiktoken.get_encoding('cl100k_base')
    if model in ['gpt-3.5-turbo-0125']:
        num_tokens = 0
        for message in messages:
            num_tokens += 4
            for key, value in message.items():
                num_tokens += len(encoding.encode(value))
                if key == 'name':
                    num_tokens -= 1
        num_tokens += 2
        return num_tokens
    else:
        raise NotImplementedError(f'''num_tokens_from_messages() is not presently implemented for model {model}.''')


# Функция дробления текста на чанки
def split_text(txt_file, chunk_size=chunk_size):
    source_chunks = []
    splitter = RecursiveCharacterTextSplitter(separators=['\n', '\n\n', '. '],
                                              chunk_size=chunk_size,
                                              chunk_overlap=0)
    for chunk in splitter.split_text(txt_file):
        source_chunks.append(Document(page_content=chunk, metadata={}))
    print(f'\n\nТекст разбит на {len(source_chunks)} чанков.')
    return source_chunks


# Функция получения ответа от модели
def answer_index(system, user, chunk, temp=temperature, model='gpt-3.5-turbo-0125'):
    messages = [
        {'role': 'system', 'content': system},
        {'role': 'user', 'content': user + f'{chunk}'}
    ]
    completion = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temp)
    print(f'{num_tokens_from_messages(messages)} токенов на чанк')
    return completion.choices[0].message.content


def process_one_file(file_path, system, user):
    with open(file_path, 'r') as txt_file:
        text = txt_file.read()
    source_chunks = split_text(text)
    processed_text = ''
    unprocessed_text = ''
    for chunk in source_chunks:
        attempt = 0
        while attempt < 3:
            try:
                answer = answer_index(system, user, chunk.page_content)
                break  # Успешно получили ответ, выходим из цикла попыток
            except Exception as e:
                attempt += 1  # Увеличиваем счетчик попыток
                print(f'\n\nПопытка {attempt} не удалась из-за ошибки: {str(e)}')
                time.sleep(10)  # Ожидаем перед следующей попыткой
                if attempt == 3:
                    answer = ''
                    print(f'\n\nОбработка элемента {chunk} не удалась после 3 попыток')
                    unprocessed_text += f'{chunk}\n\n'
        processed_text += f'{answer}\n\n'  # Добавляем ответ в результат
        # print(f'{answer}')  # Выводим ответ
    return processed_text, unprocessed_text


# Находим строки с разметкой и дублируем их без разметки
def duplicate_markdown(text):
    new_lines = []
    for line in text.split('\n'):
        if line.startswith('# '):
            new_lines.append(f'{line}.' + '\n' + f'{line[2:]}.')
        elif line.startswith('## '):
            new_lines.append(f'{line}.' + '\n' + f'{line[3:]}.')
        else:
            new_lines.append(line)
    return '\n'.join(new_lines)


# Создаем объект для разбиения текста на секции по заголовкам
markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=[("##", "Header 2")])

# Обработка Видео

In [ ]:
YouTube_video_title = "Python в вопросах и ответах" # Изучаем Python | 1 глава
video_url = 'https://youtu.be/LRuszWRN4uI' # Только 1 глава


ydl_opts = {
    'format': 'bestaudio/best',
    'postprocessors': [{
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'm4a',
        'preferredquality': '192'}],
    'outtmpl': 'python_part_1.m4a'
}
# Получаем аудио файл 'python_part_1.m4a' из видео
with youtube_dl.YoutubeDL(ydl_opts) as ydl:
    ydl.download([video_url])

In [ ]:
# transcribe
model = whisper.load_model('large')
text_from_video = model.transcribe('python_part_1.m4a')['text']

# save text_from_video
with open("text_from_video.txt", "w") as file:
    file.write(text_from_video)

100%|█████████████████████████████████████| 2.88G/2.88G [00:39<00:00, 77.2MiB/s]


In [ ]:
processed_text, unprocessed_text = process_one_file("text_from_video.txt", system, user)
# Текст разбит на 28 чанков

with open("processed_text_video.txt", "w") as file:
    file.write(processed_text)

In [ ]:
with open('processed_text_video.txt', 'r') as txt_file:
        processed_text_video = txt_file.read()

# Находим строки с разметкой и дублируем их без разметки
processed_text_video = duplicate_markdown(processed_text_video)

# Получаем список документов, разбитых по заголовкам
md_header_splits = markdown_splitter.split_text(processed_text_video)
print(len(md_header_splits))

# Формируем индексную базу
db_text_video = FAISS.from_documents(md_header_splits, embeddings)

# Сохранение базы в Колаб
index_name = "db_text_video"
db_text_video.save_local(folder_path='',
              index_name=index_name)
len(db_text_video.docstore._dict)

93


93

In [13]:
chunk = db_text_video.docstore._dict['d3f3ebb8-a051-4c6d-a921-e317d4afac79']
print(f'{chunk.metadata}\n{chunk.page_content}')

{'Header 2': 'План работы.'}
План работы.
Мы будем читать это дело по главам, на каждую главу у нас будет, скорее всего, отдельный оратор, и он будет давать презентацию, а по ходу вот этой презентации я буду задавать вопросы, каверзные и не очень, ну или подкидывать тему для обсуждения.


# Обработка Аудио

In [ ]:
audio_url = 'https://drive.google.com/file/d/1TLsB3VVMhFTdP6Iwacw5H3nce5GpO7ND/view?usp=sharing'
id = audio_url.split('/')[-2]
url = f'https://drive.google.com/uc?export=download&id={id}'
gdown.download(url, 'python_audio.m4a', quiet=True)

# transcribe
model = whisper.load_model('large')
text_from_audio = model.transcribe('python_audio.m4a')['text']

# save text_from_audio
with open("text_from_audio.txt", "w", encoding="utf-8") as file:
    file.write(text_from_audio)

In [ ]:
processed_text_audio, unprocessed_text = process_one_file("text_from_audio.txt", system, user)
# Текст разбит на 11 чанков.

with open("processed_text_audio.txt", "w", encoding="utf-8") as file:
    file.write(processed_text_audio)

In [ ]:
with open('processed_text_audio.txt', 'r') as txt_file:
        processed_text_audio = txt_file.read()

# Находим строки с разметкой и дублируем их без разметки
processed_text_audio = duplicate_markdown(processed_text_audio)

# Получаем список документов, разбитых по заголовкам
md_header_splits = markdown_splitter.split_text(processed_text_audio)
print(len(md_header_splits))

# Формируем индексную базу
db_text_audio = FAISS.from_documents(md_header_splits, embeddings)

# Сохранение базы в Колаб
index_name = "db_text_audio"
db_text_audio.save_local(folder_path='',
              index_name=index_name)
len(db_text_audio.docstore._dict)

31


31

In [12]:
chunk = db_text_audio.docstore._dict['9f201b84-87fd-419c-a591-c8b0692775c3']
print(f'{chunk.metadata}\n{chunk.page_content}')

{'Header 2': 'Проблема с управлением кодом.'}
Проблема с управлением кодом.
Но программисты очень быстро столкнулись с тем, что программы стали большими. И вот когда в такой условной программе где-то 10 тысяч строк кода, то читать такой код становится уже невозможно. И даже если сильно постараться все хорошо закомментировать и попробовать разбить этот код на файлы, все равно останется масса проблем.


# Обработка Текста

In [ ]:
text_url = 'https://docs.google.com/document/d/1GwrAY4NoqPImiSMola-53ftRh5K-3hDmivWzvuKfMfA/edit?usp=sharing'
id = text_url.split('/')[-2]
url = f'https://docs.google.com/document/d/{id}/export?format=txt'
gdown.download(url, 'python_text.txt', quiet=True)

processed_python_text, unprocessed_text = process_one_file('python_text.txt', system, user)
# Текст разбит на 10 чанков.

with open("processed_python_text.txt", "w", encoding="utf-8") as file:
    file.write(processed_python_text)

In [ ]:
with open('processed_python_text.txt', 'r', encoding="utf-8") as txt_file:
        processed_python_text = txt_file.read()

# Находим строки с разметкой и дублируем их без разметки
processed_python_text = duplicate_markdown(processed_python_text)

# Получаем список документов, разбитых по заголовкам
md_header_splits = markdown_splitter.split_text(processed_python_text)
print(len(md_header_splits))

# Формируем индексную базу
db_python_text = FAISS.from_documents(md_header_splits, embeddings)

# Сохранение базы в Колаб
index_name = "db_python_text"
db_python_text.save_local(folder_path='',
              index_name=index_name)
len(db_python_text.docstore._dict)

40


40

In [11]:
chunk = db_python_text.docstore._dict['511113a7-8f70-48bd-a6f4-30b29ba9b5c2']
print(f'{chunk.metadata}\n{chunk.page_content}')

{'Header 2': 'Название секции: Пузырьковая сортировка.'}
Название секции: Пузырьковая сортировка.
Этот простой алгоритм выполняет итерации по списку, сравнивая элементы попарно и меняя их местами, пока более крупные элементы не «всплывут» в начало списка, а более мелкие не останутся на «дне».


# Объединяю индексные базы

In [3]:
db_text_video = FAISS.load_local(folder_path='',
                                 embeddings=embeddings,
                                 index_name='db_text_video')

db_text_audio = FAISS.load_local(folder_path='',
                                 embeddings=embeddings,
                                 index_name='db_text_audio')

db_python_text = FAISS.load_local(folder_path='',
                                  embeddings=embeddings,
                                  index_name='db_python_text')

In [ ]:
db = FAISS.from_documents(db_text_video.docstore._dict.values(), embeddings)
db.merge_from(db_text_audio)
db.merge_from(db_python_text)

db.save_local(folder_path='', index_name='db')
print(len(db.docstore._dict))

164


In [15]:
db = FAISS.load_local(folder_path='',
                      embeddings=embeddings,
                      index_name='db')
print(len(db.docstore._dict))

164


In [22]:
def chunks_and_scores(query, db, doc_print=True):
    docs_and_scores = db.similarity_search_with_score(query, k=3)
    print('Docs:', [doc[0] for doc in docs_and_scores]) if doc_print else None
    print('Scores:', [doc[1] for doc in docs_and_scores])

query = 'Объектно-ориентированное программирование'
chunks_and_scores(query, db)

Docs: [Document(page_content='Объектно-ориентированное программирование.\nПотом пришло объектно-ориентированное программирование, где стали рассматриваться конкретные объекты и подходы работы с ними, такие как инкапсуляция, наследование, полиморфизм.', metadata={'Header 2': 'Объектно-ориентированное программирование.'}), Document(page_content='Отличия между парадигмами программирования.\nЯ смотрю, у тебя есть процедурное, объектно-ориентированное и функциональное программирование. Ты выделил. А в чем между ними разница вообще? Ну, соответственно, это три парадигмы программирования.', metadata={'Header 2': 'Отличия между парадигмами программирования.'}), Document(page_content='Парадигмы программирования.\nТо есть… То есть, в общем-то, три парадигмы и разница между ними – это интересно, это познавательно, это очень классно. То есть, как бы, вопрос возникает номер раз по точности формулировки, и вопрос номер два – по времени возникновения. То есть, возникло ли функциональное программирова

In [24]:
query = 'Есть ли жизнь на Марсе?'
chunks_and_scores(query, db, doc_print=False)

Scores: [0.40278316, 0.42338064, 0.42583922]


# Нейро-консультатн (вариант 1)

In [37]:
def message_content_and_scores(topic, db, k):
    docs_and_scores = db.similarity_search_with_score(topic, k=k)
    message_content = ', '.join([f'\nChunk {i+1}: {doc[0].page_content}'
                                 for i, doc in enumerate(docs_and_scores)])
    scores = [doc[1] for doc in docs_and_scores]
    return message_content, scores


def gpt_35_turbo(messages, temp=0.5):
    return client.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        messages=messages,
        temperature=temp)


def print_tokens_count(completion):
    print(f'Вопрос: {completion.usage.prompt_tokens} токенов. '
          f'\nОтвет: {completion.usage.completion_tokens} токенов. '
          f'\nВсего: {completion.usage.total_tokens} токенов.')
    print(datetime.datetime.now(), '\n')


def answer_gpt(system, topic, db, k=3, temp=0):
    message_content, scores = message_content_and_scores(topic, db, k)
    messages = [
        {"role": "system", "content": system},
        {"role": "user",
         "content":
        f"Ответь на вопрос 500 слов. Документ с информацией для ответа: {message_content}\n\nВопрос: \n{topic}"}
    ]
    completion = gpt_35_turbo(messages, temp)

    print('Symbols Message_content:', len(message_content))
    print('Scores:', scores)
    print_tokens_count(completion)
    return completion.choices[0].message.content


system = """Ты консультант по языку PYTHON и книге Марка Лутца Изучаем Python. Ответь на вопрос на
основе Документа с информацией. Не упоминай Документ с информацией при ответе. Ты не должен отвечать
на вопросы, которые не связанны с темами, содержащимися в Документе. Если будет вопрос не связанный
с темами Документа, ответь, что отвечаешь на вопросы только о языке PYTHON и книге Марка Лутца 'Изучаем Python'"""

In [30]:
topic = 'Что такое Объектно-ориентированное программирование?'
answer_gpt(system, topic, db)

Symbols Message_content: 872
Scores: [0.15657392, 0.25235257, 0.2534676]
Вопрос: 540 токенов. 
Ответ: 366 токенов. 
Всего: 906 токенов.
2024-02-22 14:55:28.152604 



'Объектно-ориентированное программирование (ООП) – это парадигма программирования, в которой основными концепциями являются объекты и классы. В рамках ООП программа рассматривается как набор взаимодействующих объектов, каждый из которых обладает определенными свойствами и методами. Основные принципы ООП включают в себя инкапсуляцию, наследование и полиморфизм.\n\nИнкапсуляция позволяет объединить данные и методы, работающие с этими данными, внутри одного объекта, скрывая детали реализации от внешнего мира. Наследование позволяет создавать новые классы на основе существующих, повторно используя их свойства и методы. Полиморфизм позволяет объектам разных классов обладать одинаковым интерфейсом для взаимодействия, при этом реализуя этот интерфейс по-разному.\n\nТаким образом, в ООП основное внимание уделяется объектам, их взаимодействию и организации программы в виде иерархии классов.'

In [38]:
topic = 'Что скажешь о Книге Марка Лутца "Изучаем Python"'
answer_gpt(system, topic, db)

Symbols Message_content: 684
Scores: [0.15492645, 0.23631567, 0.2519761]
Вопрос: 541 токенов. 
Ответ: 296 токенов. 
Всего: 837 токенов.
2024-02-22 15:01:07.621254 



'Книга Марка Лутца "Изучаем Python" является огромным двухтомником, который содержит обширную информацию о Python. В ней можно найти как достоинства, так и недостатки этого языка программирования, а также технические превосходства по сравнению с другими языками. Рекомендации по чтению указывают на то, что книга довольно подробная, но в то же время может быть немного нудной из-за своего объема и стиля изложения. Некоторые отзывы отмечают, что перевод книги может быть не самым лучшим, а некоторые моменты могут быть изложены довольно сложно. Однако, несмотря на это, книга считается одним из лучших учебников по Python, содержащим обширную информацию по базовым аспектам этого языка программирования.'

# Нейро-консультатн (вариант 2)

In [48]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

In [53]:
# сохраняем в переменную текст промпта
prompt_template = """Ты консультант по языку PYTHON и книге Марка Лутца Изучаем Python. Ответь на вопрос на
основе Документа с информацией. Не упоминай Документ с информацией при ответе. Ты не должен отвечать
на вопросы, которые не связанны с темами, содержащимися в Документе. Если будет вопрос не связанный
с темами Документа, ответь, что отвечаешь на вопросы только о языке PYTHON и книге Марка Лутца 'Изучаем Python'.
Дай ответ на 500 слов.
{context}
Question: {question}"""

PROMPT = PromptTemplate(
         template=prompt_template,
         input_variables=["context", "question"])

qa = RetrievalQA.from_chain_type(llm=ChatOpenAI(model_name="gpt-3.5-turbo-0125", temperature=0.3, max_tokens=2000),
                                 # llm=OpenAI(temperature=0.5, max_tokens=2000),
                                 retriever=db.as_retriever(# search_type="similarity",
                                                           search_type="mmr",
                                                           # search_type="similarity_score_threshold",
                                                           search_kwargs={# 'score_threshold': 0.7,
                                                                          'k': 5,
                                                                          'lambda_mult': 0.7}), # 0 - max. разнообразие
                                 chain_type_kwargs={"prompt": PROMPT})

In [55]:
text = 'Что такое Объектно-ориентированное программирование?'
qa.invoke(text)

{'query': 'Что такое Объектно-ориентированное программирование?',
 'result': 'Объектно-ориентированное программирование - это подход к разработке программного обеспечения, в котором основной упор делается на объекты, их свойства и методы. В рамках этого подхода каждый объект представляет собой экземпляр определенного класса, который определяет его структуру и поведение. Основные принципы объектно-ориентированного программирования включают в себя инкапсуляцию, наследование и полиморфизм. Инкапсуляция позволяет скрыть детали реализации объекта и предоставить к нему доступ только через определенные методы. Наследование позволяет создавать новые классы на основе уже существующих, расширяя или изменяя их функциональность. Полиморфизм позволяет использовать объекты разных классов с одинаковым интерфейсом без необходимости знать их конкретный тип. Объектно-ориентированное программирование позволяет создавать более гибкие, модульные и масштабируемые программы, облегчает повторное использование

In [54]:
text = 'Что скажешь о Книге Марка Лутца "Изучаем Python"'
qa.invoke(text)

{'query': 'Что скажешь о Книге Марка Лутца "Изучаем Python"',
 'result': 'Книга Марка Лутца "Изучаем Python" является огромным двухтомником, который содержит очень подробную информацию о Python. Некоторые разработчики могут считать ее нудной и косноязычной из-за плохого перевода. Однако, несмотря на это, книга считается одним из лучших ресурсов для изучения Python. Она может быть сложной для новичков, но предоставляет обширные знания о языке программирования. Кроме того, изучение Python является важным и рекомендуемым для всех, так как является современным и развивающимся языком программирования.'}